In [2]:
%%capture
!pip install --upgrade pyforest
!pip install XlsxWriter
from pyforest import *
from tqdm import tqdm
import xlsxwriter
import pandas as pd
from tqdm import tqdm

#Dateien Laden und vorbereiten

##Daten laden und in xlsx speichern

In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install xlsxwriter
from collections import defaultdict
from pathlib import Path
from bs4 import BeautifulSoup


     |████████████████████████████████| 148 kB 5.2 MB/s 
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "

In [ ]:



dir_path = "/content/drive/MyDrive/Masterarbeit/Data/raw_data/texte_xmls/texte"

# Textdateien laden und Dataframe mit Text und ID erstellen und speichern
results = defaultdict(list)
for file in tqdm(Path(dir_path).iterdir()):
  if '.text' in str(file):
      with open(file, 'r') as f:
        data = f.read()
      Bs_data = BeautifulSoup(data, "xml")
      results["id"].append(Bs_data.find('header').get('paula_id').replace('.text',''))
      results["text"].append(Bs_data.body.string)

texts_df = pd.DataFrame(results)
texts_df.to_excel('/content/drive/MyDrive/Masterarbeit/Data/texts.xlsx', engine='xlsxwriter')

0it [00:13, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
dir_path = "/content/drive/MyDrive/Masterarbeit/Data/raw_data/texte_xmls/metadaten"

# Metadaten Laden und in xlsx speichern
results = defaultdict(list)
for file in tqdm(Path(dir_path).iterdir()):
  if '.meta' in str(file):
      with open(file, 'r') as f:
        data = f.read()
      Bs_data = BeautifulSoup(data, "xml")
      results["id"].append(Bs_data.find('header').get('paula_id').replace('.meta',''))
      results["cerf_rating"].append(Bs_data.find('feat', {'name':'_rating_fair_cefr'}).get('value'))
      results["cerf_rating_test"].append(Bs_data.find('feat', {'name':'_test_level_cefr'}).get('value'))
      results["task_topic"].append(Bs_data.find('feat', {'name':'_task_topic'}).get('value'))

texts_df = pd.DataFrame(results)
texts_df.to_excel('/content/drive/MyDrive/Masterarbeit/Data/metadaten.xlsx', engine='xlsxwriter', index=False)

##Texte aufbereiten
(überflüssige Leerzeichen entfernen, Datum, Zahlen und €-Symbol in Platzhalter umwandeln)

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/texts.xlsx', index_col=0)  
metadaten = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/metadaten.xlsx', index_col=0)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%capture
!pip install -U spacy
!python -m spacy download de_core_news_lg
import spacy
nlp = spacy.load("de_core_news_lg")

In [ ]:
texts = data['text'].astype('str').tolist()

In [ ]:
import numpy as np
import re
texts = data['text'].astype('str').tolist()
new_texts = []
for text in texts:
  if "Sehr geehrte Damen und Herren," in text:
    match = re.compile('(Sehr geehrte Damen und Herren,\W(.|\n)*)', re.IGNORECASE)
    matchResult = match.search(text)
    if matchResult:
      text = matchResult.group(1)
  if "Anlagen:" in text:
    match = re.compile('\w+\W(.|\n)*(?=Anlagen:)', re.IGNORECASE)
    matchResult = match.search(text)
    if matchResult:
      text = matchResult.group(0)
  
  text = re.sub('\`|\"|\“|\”|\„', ' ', text)
  text = re.sub('(^\s*(3[01]|[12][0-9]|0?[1-9])\.(1[012]|0?[1-9])\.((?:19|20)\d{2})\s*$)|^\s*(3[01]|([12][0-9]|0?[1-9])\.(1[012]|0?[1-9])\.\d+)', ' -date- ', text)
  text = re.sub('((20\d\d)|(19\d\d))', ' -year- ', text)
  text = re.sub('(?i)januar\W|februar\W|märz\W|april\W|mai\W|juni\W|juli\W|august\W|september\W|oktober\W|november\W|dezemer\W', ' -month- ', text) 
  text = re.sub('(\d+)|(\d+\.\d+)', ' -number- ', text) 
  text = re.sub('€', ' Euro ', text)
  text = re.sub('\s\s+', ' ', text) 
  text = text.strip().replace('\n', ' ')
  new_texts.append(text)
data['text'] = new_texts

texts_df.to_excel('/content/drive/MyDrive/Masterarbeit/Data/texts.xlsx', engine='xlsxwriter')

NameError: ignored

###Texte und Metadaten aus txt laden (VERALTET)

In [ ]:
texts = data['correct_text'].tolist()
docs = []
for doc in tqdm(nlp.pipe(texts)):
    docs.append(doc)

In [ ]:
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm

dir_path = "/content/drive/MyDrive/Masterarbeit/Data"

# Textdateien laden und Dataframe laden
results = defaultdict(list)
for file in tqdm(Path(dir_path).iterdir()):
    with open(file, "r") as file_open:
        results["id"].append(file.name)
        results["text"].append(file_open.read())
df = pd.DataFrame(results)

In [ ]:
#Dadaframe als csv speichern
df.to_csv ('/content/drive/MyDrive/Masterarbeit/Data/data.csv', index = False, header=True)

##Daten aufbereiten (VERALTET)

In [ ]:
####### Metainformationen auslesen##############
import re
from tqdm import tqdm
for index in tqdm(data.index):
  match = re.compile('Task : (.+?) -')
  matchResult = match.search(data.at[index, "text"])
  if matchResult:
    data.at[index, "task"] = matchResult.group(1)

  match = re.compile('Mother tongue : (.+?) -')
  matchResult = match.search(data.at[index, "text"])
  if matchResult:
    data.at[index, "mother_tongue"] = matchResult.group(1)

  match = re.compile('CEFR level of test: (.+?)($|\W)')
  matchResult = match.search(data.at[index, "text"])
  if matchResult:
    data.at[index, "test_lvl"] = matchResult.group(1)

  match = re.compile('Overall CEFR rating: (.+?)($|\W)')
  matchResult = match.search(data.at[index, "text"])
  if matchResult:
    data.at[index, "cerf_total"] = matchResult.group(1)

  if "Sehr geehrte Damen und Herren," in data.at[index, "text"]:
    match = re.compile('(Sehr geehrte Damen und Herren,\W(.|\n)*)', re.IGNORECASE)
    matchResult = match.search(data.at[index, "text"])
    if matchResult:
      data.at[index, "clean_text"] = matchResult.group(1)
  else:
    match = re.compile('Learner text:\W*((.|\n)*)')
    matchResult = match.search(data.at[index, "text"])
    if matchResult:
      data.at[index, "clean_text"] = matchResult.group(1)

In [ ]:
data.head()

In [ ]:
data.drop(columns=['text'], inplace=True)

In [ ]:
#ID bereinigen
data["id"] = data["id"].str.replace(".txt", "")

In [ ]:
data_updated = data.copy()

In [ ]:
data_updated["clean_text"] = data_updated["clean_text"].replace(to_replace ='^.*\d{2}\n', value = '', regex = True)
data_updated["clean_text"] = data_updated["clean_text"].replace(to_replace ='(\n)|( {2,})', value = ' ', regex = True)

In [ ]:
data_updated.head()

In [ ]:
data_updated.to_excel('/content/drive/MyDrive/Masterarbeit/Data/data_clean.xlsx', engine='xlsxwriter')

#Erste Analysen

In [3]:
%%capture
!pip install altair
!pip install altair_saver
!pip install selenium
!apt-get install chromium-chromedriver
import altair
import altair_saver


In [4]:
texts = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/texts.xlsx', index_col=0)  
metadaten = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/metadaten.xlsx',)  

In [ ]:
metadaten

In [5]:
df_list = [texts,metadaten]
for df in df_list:
  df.sort_values(by=['id'],inplace=True)
  df.set_index('id', inplace=True)
  if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'],inplace=True)
data = pd.concat(df_list, axis=1)
data=data.reset_index()
data

,id,text,cerf_rating,cerf_rating_test,task_topic
0,1023_0001416,"Sehr geehrte Damen und Herren, ihre Anzeige in...",B2+,B2,Informationsnachfrage Au Pair-Agentur
1,1023_0001418,Müller Julia Bahnhofsstr. -number- A Stadt X A...,B2,B2,Informationsnachfrage Au Pair-Agentur
2,1023_0001419,"Sehr geehrte Damen und Herren, mit großem Inte...",B2,B2,Informationsnachfrage Au Pair-Agentur
3,1023_0001420,"Sehr geehrte Damen und Herren, ich habe Ihre A...",B2,B2,Informationsnachfrage Au Pair-Agentur
4,1023_0001422,"Abs. Frau EVA SCHMIDT BAHNHOFSTR, -number- - -...",B1+,B2,Informationsnachfrage Au Pair-Agentur
...,...,...,...,...,...
1028,1091_0000272,"Liebe Julia, -number- . -number- . -year- wie ...",A2,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1029,1091_0000273,"Liebe Julia, wie geht´s? Ich habe eine Frage f...",B1,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1030,1091_0000274,"Liebe Julia, Ich bin Michael. Wie geht´s dir. ...",B1,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1031,1091_0000275,"-date- Liebe Julia, wie gehts dir? Mir gehts g...",A2+,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...


In [6]:
data['task_topic'] = data['task_topic'].fillna('Einer Freundin nicht gebrauchte Fahrkarte anbieten')
data.cerf_rating.replace(['A2','A2+'],'A2', inplace=True)
data.cerf_rating.replace(['B1','B1+'],'B1', inplace=True)
data.cerf_rating.replace(['B2','B2+'],'B2', inplace=True)
data.cerf_rating.replace(['C1','C1+'],'C1', inplace=True)
data

,id,text,cerf_rating,cerf_rating_test,task_topic
0,1023_0001416,"Sehr geehrte Damen und Herren, ihre Anzeige in...",B2,B2,Informationsnachfrage Au Pair-Agentur
1,1023_0001418,Müller Julia Bahnhofsstr. -number- A Stadt X A...,B2,B2,Informationsnachfrage Au Pair-Agentur
2,1023_0001419,"Sehr geehrte Damen und Herren, mit großem Inte...",B2,B2,Informationsnachfrage Au Pair-Agentur
3,1023_0001420,"Sehr geehrte Damen und Herren, ich habe Ihre A...",B2,B2,Informationsnachfrage Au Pair-Agentur
4,1023_0001422,"Abs. Frau EVA SCHMIDT BAHNHOFSTR, -number- - -...",B1,B2,Informationsnachfrage Au Pair-Agentur
...,...,...,...,...,...
1028,1091_0000272,"Liebe Julia, -number- . -number- . -year- wie ...",A2,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1029,1091_0000273,"Liebe Julia, wie geht´s? Ich habe eine Frage f...",B1,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1030,1091_0000274,"Liebe Julia, Ich bin Michael. Wie geht´s dir. ...",B1,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...
1031,1091_0000275,"-date- Liebe Julia, wie gehts dir? Mir gehts g...",A2,A2,Einer Freundin nicht gebrauchte Fahrkarte anbi...


###Übersicht Anzahl Texte pro Aufgabenstellung

In [ ]:
tasks = data[['task_topic', 'cerf_rating_test']]
tasks = tasks[['task_topic', 'cerf_rating_test']].drop_duplicates()
tasks.to_excel('/content/drive/MyDrive/Masterarbeit/Data/tasks_test-lvl_overview.xlsx', engine='xlsxwriter')
tasks

In [ ]:
data[['task_topic','cerf_rating_test']].groupby(['task_topic']).agg('count')

In [ ]:
#Verteilung der Aufgabenstellung
bars = alt.Chart(data).mark_bar().encode(
  x=alt.X(
      'count()', 
      axis=alt.Axis(title='Anzahl der Wörter')
      ),
  y=alt.Y(
      'task_topic', 
      axis=alt.Axis(title='Aufgabenstellung')
      ),
  color=alt.Color(
      'task_topic:N', 
      legend=None
      ),
)
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=0 
).encode(
    text='count():Q'
)
chart = (bars + text).properties(height=300, width=500)
altair_saver.save(chart, '/content/drive/MyDrive/Masterarbeit/Abbildungen/aufgabenstellung_verteilung.png')
chart

### Texte pro Niveaustufe

In [ ]:
#Verteilung der Niveaustufen
bars = alt.Chart(data).mark_bar().encode(
  x=alt.X('cerf_rating', axis=alt.Axis(title='Niveaustufen')),
  y=alt.Y('count()', axis=alt.Axis(title='Anzahl Texte')),
  color=alt.Color('cerf_rating:O'),
)
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=-10,
    dy=-15 
).encode(
    text='count():Q'
)
chart = (bars + text).properties(height=300, width=500)
altair_saver.save(chart, '/content/drive/MyDrive/Masterarbeit/Abbildungen/niveaustufen_verteilung.png')
chart

###Zusammenhang Textlänge - Niveaustufe

In [7]:
%%capture
!pip install spacy --upgrade
import spacy
!python -m spacy download de_dep_news_trf
import spacy_transformers
!pip install textacy

In [8]:
import textacy
from textacy import text_stats
from tqdm import tqdm

In [9]:
#ata = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/data_clean_xls.xlsx', index_col=0)  
data.head()

,id,text,cerf_rating,cerf_rating_test,task_topic
0,1023_0001416,"Sehr geehrte Damen und Herren, ihre Anzeige in...",B2,B2,Informationsnachfrage Au Pair-Agentur
1,1023_0001418,Müller Julia Bahnhofsstr. -number- A Stadt X A...,B2,B2,Informationsnachfrage Au Pair-Agentur
2,1023_0001419,"Sehr geehrte Damen und Herren, mit großem Inte...",B2,B2,Informationsnachfrage Au Pair-Agentur
3,1023_0001420,"Sehr geehrte Damen und Herren, ich habe Ihre A...",B2,B2,Informationsnachfrage Au Pair-Agentur
4,1023_0001422,"Abs. Frau EVA SCHMIDT BAHNHOFSTR, -number- - -...",B1,B2,Informationsnachfrage Au Pair-Agentur


In [10]:
#Anzahl der Wörter auslesen
de = spacy.load('de_dep_news_trf')  

for index in tqdm(data.index):
  doc = textacy.make_spacy_doc(data.at[index, "text"], lang=de)
  ts = textacy.text_stats.TextStats(doc)
  data.at[index, "word_count"] = ts.n_words

100%|██████████| 1033/1033 [14:40<00:00,  1.17it/s]


In [11]:
data['word_count'].max()

367.0

In [ ]:
chart =  alt.Chart(data, width=100, height=400).mark_circle(size=20).encode(
    x=alt.X(
        'jitter:Q', 
        title=None, 
        scale=alt.Scale(domain=[-5, 5]),
        axis=alt.Axis(
            values=[0], 
            ticks=True, 
            grid=False, 
            labels=False,
            ), 

        ),
    y=alt.Y(
        'word_count:Q', 
        title='Anzahl der Wörter'
        ),
    column=alt.Column(
        'cerf_total:O', 
        title='Niveaustufen', 
        header=alt.Header(
            labelAngle=0, 
            titleOrient='top', 
            labelOrient='bottom', 
            labelAlign='center', 
            labelPadding=13),
    ),

).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=20,
).configure_view(
    stroke=None,
)

#altair_saver.save(chart, '/content/drive/MyDrive/Masterarbeit/Abbildungen/textlänge_verteilung.png')
chart

In [ ]:
selection = alt.selection_multi(fields=['task'], bind='legend')

chart =  alt.Chart(data, width=100, height=400).mark_circle(size=20).encode(
    x=alt.X(
        'jitter:Q', 
        title=None, 
        scale=alt.Scale(domain=[-5, 5]),
        axis=alt.Axis(
            values=[0], 
            ticks=True, 
            grid=False, 
            labels=False,
            ), 

        ),
    y=alt.Y(
        'word_count:Q', 
        title='Anzahl der Wörter'
        ),
    color=alt.Color(
        'task:N', 
        #legend=None
        ),
    opacity=alt.condition(
        selection, 
        alt.value(1), 
        alt.value(0.1),
        ),
    column=alt.Column(
        'cerf_total:O', 
        title='Niveaustufen', 
        header=alt.Header(
            labelAngle=0, 
            titleOrient='top', 
            labelOrient='bottom', 
            labelAlign='center', 
            labelPadding=13
            ),
        ),


).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=20,
).configure_view(
    stroke=None,
).add_selection(selection)

chart